<a href="https://colab.research.google.com/github/joeantc/Fitlist/blob/main/Fitlist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import csv
import os
import requests
import dropbox
import requests
from notion_client import Client
from datetime import datetime as dt
from concurrent.futures import ThreadPoolExecutor
import itertools


try:
    from google.colab import userdata
    APP_KEY = userdata.get('APP_KEY')
    APP_SECRET = userdata.get('APP_SECRET')
    REFRESH_TOKEN = userdata.get('REFRESH_TOKEN')
    NOTION_API_KEY = userdata.get('NOTION_API_KEY')

except ImportError:
    # Fallback for environments other than Google Colab
    APP_KEY = os.getenv('APP_KEY')
    APP_SECRET = os.getenv('APP_SECRET')
    REFRESH_TOKEN = os.getenv('REFRESH_TOKEN')
    NOTION_API_KEY = os.getenv('NOTION_API_KEY')

# Setup Dropbox API

# APP_KEY = userdata.get('APP_KEY')


CSV_FILE_PATH = '/export.csv'

# Setup Notion API

NOTION_DATABASE_ID = '11bfc863ef5f80468d10ffb86578164a'


# Initialize Notion client
notion = Client(auth=NOTION_API_KEY)


In [17]:

# Step 1: Get Authorization Code
def get_auth_token():
  print("Go to the following URL to authorize the app:")
  print(f"https://www.dropbox.com/oauth2/authorize?client_id={APP_KEY}&token_access_type=offline&response_type=code")
  authorization_code = input("Enter the authorization code here: ")

# Step 2: Exchange Authorization Code for Access Token and Refresh Token
def get_tokens():
    try:
        response = requests.post(
            "https://api.dropbox.com/oauth2/token",
            data={
                "grant_type": "authorization_code",
                "code": authorization_code,
                "client_id": APP_KEY,
                "client_secret": APP_SECRET,
            }
        )
        response.raise_for_status()
        tokens = response.json()
        print("Response from Dropbox:", tokens)
        access_token = tokens.get("access_token")
        refresh_token = tokens.get("refresh_token")

        if not refresh_token:
            print("Refresh token not found. Make sure token_access_type=offline is used in the authorization URL.")
            return None, None

        print("Access Token:", access_token)
        print("Refresh Token:", refresh_token)
        return access_token, refresh_token
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving tokens: {e}")
        print("Response content:", response.content)
        return None, None

#access_token, refresh_token = get_tokens()


In [18]:
def refresh_access_token():
    try:
        response = requests.post(
            "https://api.dropbox.com/oauth2/token",
            data={
                "grant_type": "refresh_token",
                "refresh_token": REFRESH_TOKEN,
                "client_id": APP_KEY,
                "client_secret": APP_SECRET,
            },
        )
        response.raise_for_status()
        new_token = response.json()["access_token"]
        return new_token
    except requests.exceptions.RequestException as e:
        print(f"Error refreshing access token: {e}")
        return None

# Dropbox initialization
DROPBOX_ACCESS_TOKEN = refresh_access_token()
dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)

In [19]:
# Step 1: Download CSV File from Dropbox
def download_csv_from_dropbox():
    try:
        metadata, res = dbx.files_download(CSV_FILE_PATH)
        csv_content = res.content.decode('utf-8')
        return csv_content
    except Exception as e:
        print(f"Error downloading file from Dropbox: {e}")
        return None

In [20]:
def create_records_in_notion(entries):
    # If entries is a string (CSV content), parse it into rows
    if isinstance(entries, str):
        csv_reader = csv.DictReader(entries.splitlines())
        entries = list(csv_reader)

    for row in entries:
        try:
            # Cast 'Date' column to a Notion-compatible format
            date_value = dt.strptime(row['Date'], '%d-%b-%Y').date().isoformat()
        except ValueError as e:
            print(f"Error parsing date for row {row}: {e}")
            date_value = '2023-01-01'  # Default date in case of error

        # Create new record properties, handling empty values gracefully
        new_page = {
            "parent": {"database_id": NOTION_DATABASE_ID},
            "properties": {
                'Date': {'date': {'start': date_value}},
                'Exercise': {'title': [{'text': {'content': row['Exercise']}}]} if row['Exercise'] else {'title': [{'text': {'content': 'No Exercise'}}]},
                'Set': {'number': int(row['Set #'])} if row['Set #'] else None,
                'Reps': {'number': int(row['Reps'])} if row['Reps'] else None,
                'Weight': {'number': int(row['Weight'])} if row['Weight'] else None,
                'Duration': {'number': int(row['Duration'])} if row['Duration'] else None,
                'Distance': {'rich_text': [{'text': {'content': row['Distance']}}]} if row['Distance'] else None,
                'Resistance': {'rich_text': [{'text': {'content': row['Resistance']}}]} if row['Resistance'] else None,
                'Notes': {'rich_text': [{'text': {'content': row['Notes']}}]} if row['Notes'] else None,
            }
        }

        # Remove any properties with None values
        new_page['properties'] = {k: v for k, v in new_page['properties'].items() if v is not None}

        try:
            notion.pages.create(**new_page)
            print(f"Successfully added record for {row['Date']}")
        except Exception as e:
            print(f"Error adding record to Notion for row {row}: {e}")

In [21]:
# Step 7: Query Notion Database
def query_notion_database():
    try:
        results = []
        has_more = True
        start_cursor = None
        while has_more:
            query = notion.databases.query(database_id=NOTION_DATABASE_ID, start_cursor=start_cursor)
            results.extend(query.get('results', []))
            has_more = query.get('has_more', False)
            start_cursor = query.get('next_cursor', None)
        return results
    except Exception as e:
        print(f"Error querying Notion database: {e}")
        return []

# Step 8: Compare CSV with Notion Records to Find Missing Entries
def compare_csv_with_notion(csv_content, notion_records):
    csv_reader = csv.DictReader(csv_content.splitlines())
    csv_entries = [row for row in csv_reader]

    # Create a set of unique identifiers from the Notion records (Date, Set, Exercise)
    notion_entries = set()
    for record in notion_records:
        properties = record.get('properties', {})
        date_value = properties['Date']['date']['start'] if 'Date' in properties and properties['Date'].get('date') else None
        set_value = properties['Set']['number'] if 'Set' in properties and properties['Set'].get('number') is not None else None
        exercise_value = properties['Exercise']['title'][0]['text']['content'] if 'Exercise' in properties and properties['Exercise'].get('title') else None

        if date_value and set_value is not None and exercise_value:
            notion_entries.add((date_value, set_value, exercise_value))

    # Find CSV entries that are missing in Notion
    missing_entries = [
        row for row in csv_entries
        if (dt.strptime(row['Date'], '%d-%b-%Y').date().isoformat(), int(row['Set #']), row['Exercise']) not in notion_entries
    ]

    print(f"Number of missing items: {len(missing_entries)}")
    if missing_entries:
        print("Records in CSV but not in Notion database:")

    else:
        print("All records in CSV are present in the Notion database.")
    return missing_entries

In [22]:
if __name__ == "__main__":
    csv_content = download_csv_from_dropbox()
    if csv_content:
        # Query Notion database and compare records
        notion_records = query_notion_database()
        if notion_records:
            missing_entries = compare_csv_with_notion(csv_content, notion_records)
            # Create missing records in Notion
            create_records_in_notion(missing_entries)
        else:
            # Create all records in Notion if no records exist
            csv_reader = csv.DictReader(csv_content.splitlines())
            create_records_in_notion(list(csv_reader))

Number of missing items: 0
All records in CSV are present in the Notion database.
